In [1]:
###Import standard packages###
import numpy as np
from scipy import io

###Import functions for binning data for preprocessing###
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output
import pickle

import matplotlib.pyplot as plt
import pandas as pd
from Area2_analysis.multi_area_funcs import smooth_spk, fit_and_predict, sub_and_predict, mp_fit_lag_r2, mp_sub_lag_r2, mp_multi_fit_lag_r2
import multiprocessing as mp
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from Area2_analysis.funcs import nans
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history



## S1

### Load data

In [ ]:
# foldername = "/Users/sherryan/area2_population_analysis/multi_area/"
# filename = foldername + "s1_data_raw.mat"
# s1_data_raw = io.loadmat(filename)

# spike_times=s1_data_raw['spike_times']
# acc = s1_data_raw['acc']
# vel = s1_data_raw['vels']
# times = s1_data_raw['vel_times']

# #original data has 0.01 s = 10 ms bins 
# dt = 0.01
# t_start = times[0]
# t_end = times[-1]

# spike_times = np.squeeze(spike_times)
# for i in range(spike_times.shape[0]):
#     spike_times[i]=np.squeeze(spike_times[i])

# neural_data=bin_spikes(spike_times,dt,t_start,t_end)
# gauss_width = 40 #in ms
# bin_width = dt*1000
# smth_40 = smooth_spk(neural_data, gauss_width, bin_width)
# smth_20 = smooth_spk(neural_data, 20, bin_width)

# accs_binned=bin_output(acc,times,dt,t_start,t_end)
# vels_binned=bin_output(vel,times,dt,t_start,t_end)

# with open(foldername+'proc_data_s1.pickle','wb') as f:
#     pickle.dump([smth_40,smth_20,accs_binned,vels_binned],f)

In [2]:
folder="/Users/sherryan/area2_population_analysis/multi_area/"

with open(folder+'proc_data_s1.pickle','rb') as f:
    s1_smth_40,s1_smth_20,s1_accs_binned, s1_vels_binned=pickle.load(f,encoding='latin1')
print(s1_smth_40.shape)
print(s1_smth_20.shape)
print(s1_accs_binned.shape)
print(s1_vels_binned.shape)

n_neurons = s1_smth_20.shape[1]

n_dims = 20
if not np.isnan(s1_smth_40).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(s1_smth_40)
    pca = PCA(n_components=n_dims,random_state = 42)
    smth_40_pca = pca.fit_transform(X)
    print(smth_40_pca.shape)
    print('PCA total var explained:',sum(pca.explained_variance_ratio_))
s1_40_pca = smth_40_pca

(306698, 52)
(306698, 52)
(306698, 2)
(306698, 2)


### Single-lag

In [22]:
x_field = 'PCA'
y_field = 'hand_acc'

if x_field == 'neurons':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/neurons/S1/"
    x = s1_smth_40
if x_field == 'PCA':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/PCA/S1/"
    x = s1_40_pca
if y_field == 'hand_vel':
    y = s1_vels_binned
if y_field == 'hand_acc':
    y = s1_accs_binned

lag_axis = np.arange(-300,300,20)
pool = mp.Pool(mp.cpu_count())
s1_r2_array = [pool.starmap(mp_fit_lag_r2, [(x,y,lag,10) for lag in lag_axis])][0]
pool.close()
idx_max = np.argmax(s1_r2_array)
time_max = lag_axis[idx_max]
plt.plot(lag_axis, s1_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting ' + y_field )
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_0.png', dpi = 'figure')
plt.close()

_, weights, _ = fit_and_predict(x,y,time_max,bin_size=10)
#subtract predictions with primary decoding dimensions (at time with max R2)
pool = mp.Pool(mp.cpu_count())
s1_sub_r2_array = [pool.starmap(mp_sub_lag_r2, [(x,y,lag,10,weights) for lag in lag_axis])][0]
pool.close()

plt.plot(lag_axis,s1_sub_r2_array)
plt.title('R2 score projecting out #1 t_max dim')
idx_max = np.argmax(s1_sub_r2_array)
time_max = lag_axis[idx_max]
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_1.png', dpi = 'figure')
plt.close()


### Multi-lag

In [25]:
# foldername = "/Users/sherryan/area2_population_analysis/multi_area/"
# filename = foldername + "s1_data_raw.mat"
# s1_data_raw = io.loadmat(filename)

# spike_times=s1_data_raw['spike_times']
# acc = s1_data_raw['acc']
# vel = s1_data_raw['vels']
# times = s1_data_raw['vel_times']

# #original data has 0.01 s = 10 ms bins 
# dt = 0.05 #50ms bins
# t_start = times[0]
# t_end = times[-1]

# spike_times = np.squeeze(spike_times)
# for i in range(spike_times.shape[0]):
#     spike_times[i]=np.squeeze(spike_times[i])

# neural_data_50=bin_spikes(spike_times,dt,t_start,t_end)
# accs_50_binned=bin_output(acc,times,dt,t_start,t_end)
# vels_50_binned=bin_output(vel,times,dt,t_start,t_end)

# with open(foldername+'proc_50_data_s1.pickle','wb') as f:
#     pickle.dump([neural_data_50,accs_50_binned,vels_50_binned],f)

In [2]:
folder="/Users/sherryan/area2_population_analysis/multi_area/"

with open(folder+'proc_50_data_s1.pickle','rb') as f:
    s1_neural_data_50,s1_accs_50_binned,s1_vels_50_binned=pickle.load(f,encoding='latin1')
print(s1_neural_data_50.shape)
print(s1_accs_50_binned.shape)
print(s1_vels_50_binned.shape)

bin_width = 50

n_dims = 20
if not np.isnan(s1_neural_data_50).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(s1_neural_data_50)
    pca = PCA(n_components=n_dims,random_state = 42)
    neural_data_50_pca = pca.fit_transform(X)
    print(neural_data_50_pca.shape)
    print('PCA total var explained:',sum(pca.explained_variance_ratio_))
s1_neural_data_50_pca = neural_data_50_pca

(61339, 52)
(61339, 2)
(61339, 2)
(61339, 20)
PCA total var explained: 0.48676931827230147


In [11]:
x_field = 'PCA'
y_field = 'hand_acc'

if x_field == 'neurons':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/neurons/S1/"
    x = s1_neural_data_50
if x_field == 'PCA':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/PCA/S1/"
    x = s1_neural_data_50_pca
if y_field == 'hand_vel':
    y = s1_vels_50_binned
if y_field == 'hand_acc':
    y = s1_accs_50_binned

t_before_range = range(0,301,50)
t_after_range = range(0,301,50)
s1_multi_R2s = nans([len(t_before_range),len(t_after_range)])
j,k = 0,0
for t_before in t_after_range:
    for t_after in t_after_range:
        print('Predicting with',-t_before, 'to', t_after,'ms neural data')
        bins_before= int(t_before/bin_width) #How many bins of neural data prior to the output are used for decoding
        bins_current= 1 #Whether to use concurrent time bin of neural data
        bins_after= int(t_after/bin_width) #How many bins of neural data after the output are used for decoding
        X_cov = get_spikes_with_history(x,bins_before,bins_after,bins_current)
        X_flat = X_cov.reshape(X_cov.shape[0],(X_cov.shape[1]*X_cov.shape[2]))
        print(X_flat.shape)
        X_chopped = X_flat[~np.isnan(X_flat).any(axis = 1)]
        y_chopped = y[0+bins_before:y.shape[0]-bins_after,:]
        s1_multi_R2s[j,k] = mp_multi_fit_lag_r2(X_chopped,y_chopped)
        print('R2:',s1_multi_R2s[j,k])
        k+=1
    j+=1
    k=0

fig, ax = plt.subplots()
im = ax.imshow(s1_multi_R2s)
ax.set_xlabel('Length of lagging info')
ax.set_ylabel('Length of leading info')

ax.set_xticks(np.arange(len(t_after_range)))
ax.set_yticks(np.arange(len(t_before_range)))
ax.set_xticklabels(labels=t_after_range)
ax.set_yticklabels(labels=t_before_range)

ax.set_title('R2 predicting ' + y_field +' with different lagging/leading info')
fig.tight_layout()
for i in range(len(t_before_range)):
    for j in range(len(t_after_range)):
        text = ax.text(j, i, str(int(s1_multi_R2s[i, j]*1000)/1000),
                        ha="center", va="center", color="w")

plt.tight_layout()
plt.savefig(figDir + y_field + '_multi_lag_.png', dpi = 'figure')
plt.close()

Predicting with 0 to 0 ms neural data
(61339, 20)
R2: 0.08129166045406244
Predicting with 0 to 50 ms neural data
(61339, 40)
R2: 0.20007120523457256
Predicting with 0 to 100 ms neural data
(61339, 60)
R2: 0.2573699027208781
Predicting with 0 to 150 ms neural data
(61339, 80)
R2: 0.27596734988481497
Predicting with 0 to 200 ms neural data
(61339, 100)
R2: 0.2862983749398812
Predicting with 0 to 250 ms neural data
(61339, 120)
R2: 0.2923957032277533
Predicting with 0 to 300 ms neural data
(61339, 140)
R2: 0.2958720463954052
Predicting with 0 to 350 ms neural data
(61339, 160)
R2: 0.2983368756436272
Predicting with 0 to 400 ms neural data
(61339, 180)
R2: 0.2999474473798721
Predicting with 0 to 450 ms neural data
(61339, 200)
R2: 0.30066696609770416
Predicting with 0 to 500 ms neural data
(61339, 220)
R2: 0.3009058966097744
Predicting with -50 to 0 ms neural data
(61339, 40)
R2: 0.17533898129492664
Predicting with -50 to 50 ms neural data
(61339, 60)
R2: 0.29665080309124936
Predicting wit

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_46561/1936292039.py:53: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


## M1

### Load data

In [ ]:
# foldername = "/Users/sherryan/area2_population_analysis/multi_area/"
# filename = foldername + "m1_data_raw.mat"
# m1_data_raw = io.loadmat(filename)

# spike_times=m1_data_raw['spike_times'][:70,:] #only first 70 are M1 neurons
# acc = m1_data_raw['acc']
# vel = m1_data_raw['vels']
# times = m1_data_raw['vel_times']

# #original data has 0.001 s = 1 ms bins 
# dt = 0.01
# t_start = times[0]
# t_end = times[-1]

# spike_times = np.squeeze(spike_times)
# for i in range(spike_times.shape[0]):
#     spike_times[i]=np.squeeze(spike_times[i])

# neural_data=bin_spikes(spike_times,dt,t_start,t_end)
# gauss_width = 40 #in ms
# bin_width = dt*1000
# smth_40 = smooth_spk(neural_data, gauss_width, bin_width)
# smth_20 = smooth_spk(neural_data, 20, bin_width)

# accs_binned=bin_output(acc,times,dt,t_start,t_end)
# vels_binned=bin_output(vel,times,dt,t_start,t_end)

# with open(foldername+'proc_data_m1.pickle','wb') as f:
#     pickle.dump([smth_40, smth_20,accs_binned,vels_binned],f)

In [20]:
folder="/Users/sherryan/area2_population_analysis/multi_area/"

with open(folder+'proc_data_m1.pickle','rb') as f:
    m1_smth_40,m1_smth_20,m1_accs_binned, m1_vels_binned=pickle.load(f,encoding='latin1')
print(m1_smth_40.shape)
print(m1_smth_20.shape)
print(m1_accs_binned.shape)
print(m1_vels_binned.shape)

n_neurons = m1_smth_20.shape[1]

n_dims = 20
if not np.isnan(m1_smth_40).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(m1_smth_40)
    pca = PCA(n_components=n_dims,random_state = 42)
    smth_40_pca = pca.fit_transform(X)
    print(smth_40_pca.shape)
    print('PCA total var explained:',sum(pca.explained_variance_ratio_))
m1_40_pca = smth_40_pca

(126499, 70)
(126499, 70)
(126499, 2)
(126499, 2)
(126499, 20)
PCA total var explained: 0.5520019353069091


### Single-lag

In [24]:
x_field = 'PCA'
y_field = 'hand_vel'

if x_field == 'neurons':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/neurons/M1/"
    x = m1_smth_40
if x_field == 'PCA':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/PCA/M1/"
    x = m1_40_pca
if y_field == 'hand_vel':
    y = m1_vels_binned
if y_field == 'hand_acc':
    y = m1_accs_binned

lag_axis = np.arange(-300,300,20)
pool = mp.Pool(mp.cpu_count())
m1_r2_array = [pool.starmap(mp_fit_lag_r2, [(x,y,lag,10) for lag in lag_axis])][0]
pool.close()

idx_max = np.argmax(m1_r2_array)
time_max = lag_axis[idx_max]
plt.plot(lag_axis, m1_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting ' + y_field )
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_0.png', dpi = 'figure')
plt.close()

_, weights, _ = fit_and_predict(x,y,time_max,bin_size=10)
#subtract predictions with primary decoding dimensions (at time with max R2)
pool = mp.Pool(mp.cpu_count())
m1_sub_r2_array = [pool.starmap(mp_sub_lag_r2, [(x,y,lag,10,weights) for lag in lag_axis])][0]
pool.close()
plt.plot(lag_axis,m1_sub_r2_array)
plt.title('R2 score projecting out #1 t_max dim')
idx_max = np.argmax(m1_sub_r2_array)
time_max = lag_axis[idx_max]
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + y_field +'_1.png', dpi = 'figure')
plt.close()


### Multi-lag

In [4]:
# foldername = "/Users/sherryan/area2_population_analysis/multi_area/"
# filename = foldername + "m1_data_raw.mat"
# m1_data_raw = io.loadmat(filename)

# spike_times=m1_data_raw['spike_times']
# acc = m1_data_raw['acc']
# vel = m1_data_raw['vels']
# times = m1_data_raw['vel_times']

# #original data has 0.01 s = 10 ms bins 
# dt = 0.05 #50ms bins
# t_start = times[0]
# t_end = times[-1]

# spike_times = np.squeeze(spike_times)
# for i in range(spike_times.shape[0]):
#     spike_times[i]=np.squeeze(spike_times[i])

# neural_data_50=bin_spikes(spike_times,dt,t_start,t_end)
# accs_50_binned=bin_output(acc,times,dt,t_start,t_end)
# vels_50_binned=bin_output(vel,times,dt,t_start,t_end)

# with open(foldername+'proc_50_data_m1.pickle','wb') as f:
#     pickle.dump([neural_data_50,accs_50_binned,vels_50_binned],f)

In [5]:
folder="/Users/sherryan/area2_population_analysis/multi_area/"

with open(folder+'proc_50_data_m1.pickle','rb') as f:
    m1_neural_data_50,m1_accs_50_binned,m1_vels_50_binned=pickle.load(f,encoding='latin1')
print(m1_neural_data_50.shape)
print(m1_accs_50_binned.shape)
print(m1_vels_50_binned.shape)

bin_width = 50

n_dims = 20
if not np.isnan(m1_neural_data_50).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(m1_neural_data_50)
    pca = PCA(n_components=n_dims,random_state = 42)
    neural_data_50_pca = pca.fit_transform(X)
    print(neural_data_50_pca.shape)
    print('PCA total var explained:',sum(pca.explained_variance_ratio_))
m1_neural_data_50_pca = neural_data_50_pca

(25299, 164)
(25299, 2)
(25299, 2)
(25299, 20)
PCA total var explained: 0.2596326642721701


In [10]:
x_field = 'PCA'
y_field = 'hand_acc'

if x_field == 'neurons':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/neurons/M1/"
    x = m1_neural_data_50
if x_field == 'PCA':
    figDir = "/Users/sherryan/area2_population_analysis/figures/multi/PCA/M1/"
    x = m1_neural_data_50_pca
if y_field == 'hand_vel':
    y = m1_vels_50_binned
if y_field == 'hand_acc':
    y = m1_accs_50_binned

t_before_range = range(0,301,50)
t_after_range = range(0,301,50)
m1_multi_R2s = nans([len(t_before_range),len(t_after_range)])
j,k = 0,0
for t_before in t_after_range:
    for t_after in t_after_range:
        print('Predicting with',-t_before, 'to', t_after,'ms neural data')
        bins_before= int(t_before/bin_width) #How many bins of neural data prior to the output are used for decoding
        bins_current= 1 #Whether to use concurrent time bin of neural data
        bins_after= int(t_after/bin_width) #How many bins of neural data after the output are used for decoding
        X_cov = get_spikes_with_history(x,bins_before,bins_after,bins_current)
        X_flat = X_cov.reshape(X_cov.shape[0],(X_cov.shape[1]*X_cov.shape[2]))
        print(X_flat.shape)
        X_chopped = X_flat[~np.isnan(X_flat).any(axis = 1)]
        y_chopped = y[0+bins_before:y.shape[0]-bins_after,:]
        m1_multi_R2s[j,k] = mp_multi_fit_lag_r2(X_chopped,y_chopped)
        print('R2:',m1_multi_R2s[j,k])
        k+=1
    j+=1
    k=0

fig, ax = plt.subplots()
im = ax.imshow(m1_multi_R2s)
ax.set_xlabel('Length of lagging info')
ax.set_ylabel('Length of leading info')

ax.set_xticks(np.arange(len(t_after_range)))
ax.set_yticks(np.arange(len(t_before_range)))
ax.set_xticklabels(labels=t_after_range)
ax.set_yticklabels(labels=t_before_range)

ax.set_title('R2 predicting ' + y_field +' with different lagging/leading info')
fig.tight_layout()
for i in range(len(t_before_range)):
    for j in range(len(t_after_range)):
        text = ax.text(j, i, str(int(m1_multi_R2s[i, j]*1000)/1000),
                        ha="center", va="center", color="w")

plt.tight_layout()
plt.savefig(figDir + y_field + '_multi_lag.png', dpi = 'figure')
plt.close()

Predicting with 0 to 0 ms neural data
(25299, 20)
R2: 0.12739857372050623
Predicting with 0 to 50 ms neural data
(25299, 40)
R2: 0.1514747962418208
Predicting with 0 to 100 ms neural data
(25299, 60)
R2: 0.16549421271656062
Predicting with 0 to 150 ms neural data
(25299, 80)
R2: 0.17790198715031302
Predicting with 0 to 200 ms neural data
(25299, 100)
R2: 0.18584209551058473
Predicting with 0 to 250 ms neural data
(25299, 120)
R2: 0.18956395063798614
Predicting with 0 to 300 ms neural data
(25299, 140)
R2: 0.19166312636213567
Predicting with 0 to 350 ms neural data
(25299, 160)
R2: 0.19322630067884117
Predicting with 0 to 400 ms neural data
(25299, 180)
R2: 0.19517098879416694
Predicting with 0 to 450 ms neural data
(25299, 200)
R2: 0.1965687356692708
Predicting with 0 to 500 ms neural data
(25299, 220)
R2: 0.19752414667398255
Predicting with -50 to 0 ms neural data
(25299, 40)
R2: 0.21109251465470147
Predicting with -50 to 50 ms neural data
(25299, 60)
R2: 0.23258533576814178
Predictin

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_46561/1474082026.py:53: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
